In [ ]:
import sys
import pygame
import time
import math
from tool import scale_image, blit_rotate_center, blit_text_center
pygame.font.init()

grass = scale_image(pygame.image.load("imgs/grass.jpg"), 2.5)
track = scale_image(pygame.image.load("imgs/track.png"), 0.9)

track_border = scale_image(pygame.image.load("imgs/track-border.png"), 0.9)
track_border_mask = pygame.mask.from_surface(track_border)

finish = pygame.image.load("imgs/finish.png")
finish_mask = pygame.mask.from_surface(finish)
finish_position = (130, 250)

car = scale_image(pygame.image.load("imgs/green-car.png"), 0.55)

width, height = track.get_width(), track.get_height()
end = pygame.display.set_mode((width, height))
pygame.display.set_caption("Simulation")

font = pygame.font.SysFont("comicsans", 44)

fps = 60
path = [(175, 119), (110, 70), (56, 133), (70, 481), (318, 731), (404, 680), (418, 521), (507, 475), (600, 551), (613, 715), (736, 713),
        (734, 399), (611, 357), (409, 343), (433, 257), (697, 258), (738, 123), (581, 71), (303, 78), (275, 377), (176, 388), (178, 260)]


class GameInfo:

    def __init__(self, level=1):
        self.level = level
        self.started = False
        self.level_start_time = 0

    def reset(self):
        self.level = 1
        self.started = False
        self.level_start_time = 0

    def game_finished(self):
        return self.level > 1


class AbstractCar:
    def __init__(self, max_vel, rotation_vel):
        self.img = self.img
        self.max_vel = max_vel
        self.vel = 0
        self.rotation_vel = rotation_vel
        self.angle = 0
        self.x, self.y = self.start_pos
        self.acceleration = 0.1

    def rotate(self, left=False, right=False):
        if left:
            self.angle += self.rotation_vel
        elif right:
            self.angle -= self.rotation_vel

    def draw(self, end):
        blit_rotate_center(end, self.img, (self.x, self.y), self.angle)

    def move_forward(self):
        self.vel = min(self.vel + self.acceleration, self.max_vel)
        self.move()

    def move_backward(self):
        self.vel = max(self.vel - self.acceleration, -self.max_vel/2)
        self.move()

    def move(self):
        radians = math.radians(self.angle)
        vertical = math.cos(radians) * self.vel
        horizontal = math.sin(radians) * self.vel

        self.y -= vertical
        self.x -= horizontal

    def collide(self, mask, x=0, y=0):
        car_mask = pygame.mask.from_surface(self.img)
        offset = (int(self.x - x), int(self.y - y))
        poi = mask.overlap(car_mask, offset)
        return poi

    def reset(self):
        self.x, self.y = self.start_pos
        self.angle = 0
        self.vel = 0


class ComputerCar(AbstractCar):
    img = car
    start_pos = (150, 200)

    def __init__(self, max_vel, rotation_vel, path=[]):
        super().__init__(max_vel, rotation_vel)
        self.path = path
        self.current_point = 0
        self.vel = max_vel

    def draw_points(self, win):
        for point in self.path:
            pygame.draw.circle(end, (255, 0, 0), point, 5)

    def draw(self, end):
        super().draw(end)

    def calculate_angle(self):
        target_x, target_y = self.path[self.current_point]
        x_diff = target_x - self.x
        y_diff = target_y - self.y

        if y_diff == 0:
            desired_radian_angle = math.pi / 2
        else:
            desired_radian_angle = math.atan(x_diff / y_diff)

        if target_y > self.y:
            desired_radian_angle += math.pi

        difference_in_angle = self.angle - math.degrees(desired_radian_angle)
        if difference_in_angle >= 180:
            difference_in_angle -= 360

        if difference_in_angle > 0:
            self.angle -= min(self.rotation_vel, abs(difference_in_angle))
        else:
            self.angle += min(self.rotation_vel, abs(difference_in_angle))

    def update_path_point(self):
        target = self.path[self.current_point]
        rect = pygame.Rect(
            self.x, self.y, self.img.get_width(), self.img.get_height())
        if rect.collidepoint(*target):
            self.current_point += 1

    def move(self):
        if self.current_point >= len(self.path):
            return

        self.calculate_angle()
        self.update_path_point()
        super().move()


def draw(end, images, computer_car, game_info):
    for img, pos in images:
        end.blit(img, pos)

    computer_car.draw(end)
    pygame.display.update()


def handle_collision(computer_car, game_info):

    computer_finish_poi_collide = computer_car.collide(
        finish_mask, *finish_position)
    if computer_finish_poi_collide != None:
        blit_text_center(end, font, "Finish")
        pygame.display.update()
        pygame.time.wait(5000)
        game_info.reset()
        computer_car.reset()

class Stage2:
    def car_run(self, queueS1, queueS2):
        run = True
        clock = pygame.time.Clock()
        images = [(grass, (0, 0)), (track, (0, 0)),
                  (finish, finish_position), (track_border, (0, 0))]
        computer_car = ComputerCar(2, 4, path)
        game_info = GameInfo()
        
            
        while run:
            clock.tick(fps)

            draw(end, images, computer_car, game_info)

            while not game_info.started:
                blit_text_center(
                    end, font, f"Press any key to start")
                pygame.display.update()
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        break

                    if event.type == pygame.KEYDOWN:
                        game_info.start_level()

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    run = False
                    break

            computer_car.move()

            handle_collision(computer_car, game_info)

            if game_info.game_finished():
                blit_text_center(end, font, "Finish")
                pygame.time.wait(5000)
                game_info.reset()
                computer_car.reset()

            msg = queueS1.get()    # wait till there is a msg from s1
            if msg == 'stop':
                break # ends loop
                
        pygame.quit()